In [2]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import csv
import pathlib

# Create Chroma DB client
client = chromadb.PersistentClient(path="./chroma_db_data")

# Load CSV data
documents = []
csv_file = pathlib.Path("./6bca48b1-fine_food_reviews.csv")

if csv_file.exists():
    with open(csv_file, "r") as f:
        reader = csv.reader(f)
        header = next(reader)
        documents = [row for row in reader]
else:
    print(f"Error: File '{csv_file}' does not exist.")
    raise FileNotFoundError

# Create Chroma collection
try:
    chroma_collection = client.get_collection("reviews")
except ValueError:
    chroma_collection = client.create_collection(name="reviews")

# Create ChromaVectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Initialize Ollama
llm = Ollama(model="mixtral")

# Create HuggingFaceEmbedding object
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Create settings instance
settings_class = Settings
settings_instance = settings_class()  # Assuming the Settings class has a no-arg constructor

# Set the llm and embed_model attributes
settings_instance.llm = llm
settings_instance.embed_model = embed_model

# Create VectorStoreIndex and query engine
index = VectorStoreIndex.from_documents(documents, vector_store=vector_store, settings=settings_instance)

# Perform a query and print the response
response = index.query("What are the thoughts on food quality?")
print(response)

# Create VectorStoreIndex and query engine with a similarity threshold of 20
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, settings=settings_instance, similarity_top_k=20)

# Perform a query and print the response
response = index.query("What are the thoughts on food quality?")
print(response)

TypeError: '_Settings' object is not callable